In [17]:
import numpy as np
from numpy import loadtxt
import pandas as pd
import csv
import math
import warnings
warnings.filterwarnings('ignore')

In [18]:
#Importing training data as X_train and Y_train
file = open('dataset.csv')
csvreader = csv.reader(file, delimiter = ",")
data = []
for row in csvreader:
    data.append(row)

train_data = []
for i in range(0 , len(data)):
    col= []
    for j in range(0,len(data[i])):
        if j==len(data[i])-1:
            col.append(data[i][j])
        else:
            col.append(data[i][j])
    train_data.append(col)
file.close()
X_train = []
Y_train = []
for i in range(len(train_data)):
    cols = []
    for j in range(0,len(train_data[i])):
        if(j==len(train_data[i])-1):
            Y_train.append(train_data[i][j])
        else:
            cols.append(train_data[i][j])
    X_train.append(cols)   
for i in range(0,len(X_train)):
    for j in range(0,len(X_train[i])):
        X_train[i][j]=float(X_train[i][j])
X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_train_df = pd.DataFrame(X_train, columns = ['Height','Weight','Diameter','Color'])
Y_train_df = pd.DataFrame(Y_train,columns = ['Material'])

In [19]:
#Taking out test dataset as X_test and Y_test
test_data = []
c = 0
p = 0
m = 0
k = 0
x = []
for i in range(0,len(train_data)):
    if(train_data[i][4]==' Ceramic' and c<6):
        x.append(i)
        c=c+1
        test_data.append(train_data[i])
    if(train_data[i][4]==' Plastic' and p<6):
        x.append(i)
        p=p+1
        test_data.append(train_data[i])
    if(train_data[i][4]==' Metal' and m<6):
        x.append(i)
        m=m+1
        test_data.append(train_data[i])
    if(p>=6 and c>=6 and m>=6):
            train_data.append(train_data[i])
test_df = pd.DataFrame(data = test_data)
X_test = test_df.drop([4],axis = 'columns')
Y_test = test_df[4]
X_test = np.array(X_test).astype(np.float)
Y_test = np.array(Y_test)


In [20]:
#Deleting the Test Data set form the training set
X_train_df= X_train_df.drop(X_train_df.index[x])
Y_train_df = Y_train_df.drop(Y_train_df.index[x])


In [21]:
X_train = np.array(X_train_df).astype(np.float)
Y_train  = np.array(Y_train_df)

In [22]:
#Intializing the functions for the softmax
def initialize_weights(X,Y):
    z = np.ones((len(X[1]),np.max(Y)+1))
    return z

def set_labels_int(Y):
    for i in range(0,len(Y)):
        if Y[i] ==" Metal":
            Y[i]=0
        if Y[i] == " Plastic":
            Y[i]=1
        if Y[i] == " Ceramic":
            Y[i]=2
    return Y

def reset_labels(Y):
    y=[]
    for i in range(0,len(Y)):
        if Y[i] == 0:
            y.append(" Metal")
        if Y[i] == 1:
            y.append(" Plastic")
        if Y[i] == 2:
            y.append(" Ceramic")
    return y

def set_one_hot(Y_train): 
    y_train=[]
    for i in range(0,len(Y_train)):
        arr = []
        for j in range(0,np.max(Y_train)+1):
            if Y_train[i] == j:
                arr.append(1)
            else:
                arr.append(0)
        y_train.append(arr)
    return y_train

def training(x,y,l_r,epochs):
    w = np.ones((len(x[1]),np.max(y)+1))
    y = set_one_hot(y)
    for i in range(0,epochs):
        z = np.dot(x,w)
        z = np.exp(z)
        v = np.sum(z,axis=1)
        sm = []
        for i in range(0,len(z)):
            cols = []
            for j in range(0,len(z[i])):
                cols.append(z[i][j]/v[i])
            sm.append(cols)
        sm = np.array(sm)
        sm = y - sm
        sm = np.dot(x.T,sm)
        w = w + l_r*sm
    return w
def predict(X_test,tw):
    re = np.dot(X_test,tw)
    re = np.exp(re)
    re = re / np.sum(re, axis=1, keepdims=True)
    return np.argmax(re,axis=1)
def accuracy(y_pred,y):
    c=0
    for i in range(0,len(y)):
        if y_pred[i]==y[i]:
            c=c+1
    c = (c/len(y_pred))*100
    return c
Y_train = set_labels_int(Y_train).astype(np.int)

# Accuracy without Boosting

In [23]:
y = set_labels_int(Y_train).astype(np.int)
tw = training(X_train,y,0.1,10000)
y_pred = predict(X_test,tw)
z = set_labels_int(Y_test).astype(np.int)
print("Predicted_output")
print(y_pred)
print("Desired_output ")
print(z)
print("Accuracy")
a = accuracy(y_pred,Y_test)
print(a)

Predicted_output
[2 0 2 2 1 1 1 1 1 1 1 1 1 1 1 1 2 2]
Desired_output 
[2 0 2 2 1 1 1 1 1 1 2 0 0 0 0 0 2 2]
Accuracy
66.66666666666666


In [50]:
def checking_outputs(y,y_pred):
    x = []
    for i in range(0,len(y)):
        if y[i] != y_pred[i]:
            x.append(i)
    return x   

def model_weight(mc,w):
    error = 0
    for mcs in mc:
        error = w[mcs]+(error)
    error = ((1-error)/(error+0.0001))
    error = 0.5*np.log(error)
    return error

def updated_weight(mc,alpha,w):
    for i in range(0,len(w)):
        if i in mc:
            w[i] = w[i]*np.exp(alpha)
        else:
            w[i] = w[i]*np.exp(-alpha)
    nw = w/np.sum(w)
    return nw 

def new_dataset(nw,mc):
    new_indices = []
    ug = []
    lg = []
    c=0;
    for i in range(0,len(nw)):
        ug.append(c)
        c=c+nw[i]
        lg.append(c)
    for i in range(0,len(nw)):
        a = np.random.random()
        for i in range(0,len(ug)):
            if a>ug[i] and a<lg[i]:
                new_indices.append(i)
    new_indices = np.array(new_indices)
    return new_indices


# Implementing the Boosting Function 

In [51]:
def boosting(itr,X,Y):
    weights = []
    alphas = []
    w = np.ones(len(X))
    w = w/np.shape(X)[0]
    for i in range(0,itr):
        tw = training(X,Y,0.1,10000)
        weights.append(tw)
        y_pred = predict(X,tw)
        mc = checking_outputs(y_pred,Y)
        alpha = model_weight(mc,w)
        alphas.append(alpha) 
        nw = updated_weight(mc,alpha,w)
        new_indices = new_dataset(nw,mc)
        new_X = []
        new_Y = []
        for k in range(0,len(new_indices)):
            new_X.append(X[new_indices[k]])
            new_Y.append(Y[new_indices[k]])
            
        new_X = np.array(new_X)
        new_Y = np.array(new_Y)
        X = new_X
        Y = new_Y
    return weights,alphas


# Boosting for 10 iterations

In [52]:
we,alp = boosting(10,X_train,Y_train) 
print(we)
print(alp)

[array([[-125.49636826,  352.18439792, -223.68802966],
       [ -14.09360874,  388.24042111, -371.14681237],
       [  49.02418849, -164.36763404,  118.34344555],
       [   1.73547038,   -2.52373026,    3.78825988]]), array([[ -73.39259294,  375.3165754 , -298.92398246],
       [  33.32618942,  388.74137377, -419.0675632 ],
       [  -1.39774916, -150.97015497,  155.36790413],
       [  -3.93353366,   -0.83499475,    7.76852841]]), array([[ -37.86238091,  299.12848189, -258.26610098],
       [ 160.70744019,  158.55889759, -316.26633778],
       [ -63.32167901,  -82.35338194,  148.67506095],
       [  -2.88562344,   -1.31696125,    7.20258468]]), array([[-1.94488962e+01,  2.29774935e+02, -2.07326039e+02],
       [ 1.37535556e+02,  9.60803085e+01, -2.30615865e+02],
       [-5.02231858e+01, -6.79749304e+01,  1.21198116e+02],
       [ 5.10757802e+00,  1.23649572e-01, -2.23122759e+00]]), array([[  36.83469493,  141.08611424, -174.92080917],
       [ 267.35542367,   43.31636178, -307.671785

In [53]:
y_pred = []
for i in range(0,len(we)):
    b = predict(X_test,we[i])
    y_pred.append(b)
h=[]
for i in range(0,len(alp)):
    h.append(alp[i]*y_pred[i])
h = np.sum(h,axis=0)
h=np.sign(h)
a_10 = accuracy(h,Y_test)
print("Every classifier prediction")
print(y_pred)
print("Final Prediction after multiplying with alpha and applying sign function")
print(h)


Every classifier prediction
[array([2, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2]), array([2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2]), array([2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2]), array([2, 0, 2, 2, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 2, 0]), array([2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2]), array([2, 1, 2, 2, 0, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1]), array([0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]), array([2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])]
Final Prediction after multiplying with alpha and applying sign function
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


# Boosting for 25

In [28]:
we,alp = boosting(25,X_train,Y_train) 

In [29]:
y_pred = []
for i in range(0,len(we)):
    b = predict(X_test,we[i])
    y_pred.append(b)
h=[]
for i in range(0,len(alp)):
    h.append(alp[i]*y_pred[i])
h = np.sum(h,axis=0)
h=np.sign(h)
a_25 = accuracy(h,Y_test)
print("Every classifier prediction")
print(y_pred)
print("Final Prediction after multiplying with alpha and applying sign function")
print(h)

Every classifier prediction
[array([2, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2]), array([0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]), array([2, 2, 2, 2, 1, 1, 0, 0, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2]), array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]), array([2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2]), array([2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1]), array([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0]), array([2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2]), array([0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), array([2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2]), array([2, 0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0]), array([2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2]), array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 1, 1

# Boosting for 50

In [30]:
we,alp = boosting(50,X_train,Y_train) 

In [16]:
y_pred = []
for i in range(0,len(we)):
    b = predict(X_test,we[i])
    y_pred.append(b)
h=[]
for i in range(0,len(alp)):
    h.append(alp[i]*y_pred[i])
h = np.sum(h,axis=0)
h=np.sign(h)
a_50 = accuracy(h,Y_test)
print("Every classifier prediction")
print(y_pred)
print("Final Prediction after multiplying with alpha and applying sign function")
print(h)

Every classifier prediction
[array([2, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2]), array([2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1]), array([2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2]), array([0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 0]), array([2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0]), array([2, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 0]), array([2, 0, 2, 2, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0]), array([2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2]), array([2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2]), array([0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1]), array([2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0]), array([2, 2, 2, 2, 0, 1, 1, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2]), array([2, 0, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0]), array([1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 1, 2]), array([0, 0, 0, 0, 0, 0, 0

# Outcome

In [16]:
print("Error Rate for 10 iterations "+ str(100-a_10))
print("Error Rate for 25 iterations "+ str(100-a_25))
print("Error Rate for 50 iterations "+ str(100-a_50))

Error Rate for 10 iterations 66.66666666666667
Error Rate for 25 iterations 66.66666666666667
Error Rate for 50 iterations 66.66666666666667


In [17]:
print("Error Rate without Boosting "+str(100-a))

Error Rate without Boosting 33.33333333333334


# Observation 

We have error rate in boosting more than without boosting

# Note

## Due to more epochs it will take time to run

In [44]:

print(a)

0.003396053424117339
